*Author: acarballo <br> Latest update: March 2021 <br> Version for Google Colab*

**Setup**

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Clone TensorFlow Garden

In [ ]:
%cd '/content/gdrive/My Drive/TensorFlow'
!git clone https://github.com/tensorflow/models.git

Installing the Object Detection API

In [ ]:
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Import dependencies

In [ ]:
!pip install tf_slim
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

Base imports

In [ ]:
import os
import pathlib
import tarfile
import urllib.request
import numpy as np

import tensorflow as tf

print('TF Version:', tf.__version__)

**Create data directory and set paths**

In [ ]:
%cd '/content/'

In [ ]:
DIR = os.path.join(os.getcwd(), 'sign_language')
ANNOTATION_PATH = os.path.join(DIR, 'annotations')
TRAIN_PATH = os.path.join(DIR, 'train')
TEST_PATH = os.path.join(DIR, 'test')
MODELS_PATH = os.path.join(DIR, 'models')

In [ ]:
for path in [DIR, ANNOTATION_PATH, TRAIN_PATH, TEST_PATH, MODELS_PATH]:
  if not os.path.exists(path):
    os.makedirs(path)

**Download the model from TensorFlow Zoo**

Model info

In [ ]:
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_PATH, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_PATH, os.path.join(MODEL_NAME, 'checkpoints/'))
PATH_TO_CFG = os.path.join(MODELS_PATH, os.path.join(MODEL_NAME, 'pipeline.config'))

In [ ]:
if not os.path.exists(PATH_TO_CKPT):
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_PATH)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Model downloaded')

**Create label map**

In [ ]:
labels = [ {'name' : 'Hello', 'id' : 1}, {'name' : 'How are', 'id' : 2},
           {'name' : 'Thank you', 'id' : 3}, {'name' : 'You', 'id' : 4},
           {'name' : 'Fine', 'id' : 5} ]

with open(ANNOTATION_PATH + '/signs_label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

**Create TF Records**

*Upload generated images and PASCAL VOC labels (on local computer) <br> LabelImg: https://github.com/tzutalin/labelImg*

In [ ]:
IMAGE_PATH = '/content/sign_language/'
SCRIPTS_PATH = '/content/gdrive/MyDrive/TensorFlow/models/research/'

!python {SCRIPTS_PATH + 'generate_tfrecord.py'} -x {IMAGE_PATH + 'train'} -l {ANNOTATION_PATH + '/signs_label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + 'generate_tfrecord.py'} -x {IMAGE_PATH + 'test'} -l {ANNOTATION_PATH + '/signs_label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

**Update pipeline for transfer learning**

Import dependencies

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.protos import pipeline_pb2
from object_detection.builders import model_builder
from google.protobuf import text_format

Setup pipilene

In [ ]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PATH_TO_CFG, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PATH_TO_CKPT, 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/signs_label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/signs_label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(PATH_TO_CFG, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

**Train Model**

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
!python /content/gdrive/MyDrive/TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=/content/sign_language/models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8 --pipeline_config_path=/content/sign_language/models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --num_train_steps=5000

**Download project**

Create zip file

In [ ]:
!zip -r /content/sign_language.zip /content/sign_language

Download zip file

In [ ]:
from google.colab import files

files.download("/content/sign_language.zip")

***Hereinafter, on local computer***

Load pipeline config and build a detection model

In [ ]:
PATH_TO_CFG = 'YOUR PATH FOR PIPELINE CONFIG'
PATH_TO_CKPT = 'YOUR PATH FOR THE LATEST CHECKPOINT'    

In [ ]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

Restore checkpoint

In [ ]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(PATH_TO_CKPT).expect_partial()

Real-time detection (*won't work on google colab*)

In [ ]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np


ANNOTATION_PATH = 'YOUR PATH FOR LABEL MAP pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH, use_display_name=True)

# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    # Read frame from camera
    ret, frame = cap.read()

    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=1,
            min_score_thresh=.6,
            agnostic_mode=False)

    # Display output
    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()